In [25]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn
import pprint

pp = pprint.PrettyPrinter(indent = 4)
sess = tf.InteractiveSession()

In [26]:
h = [1,0,0,0]
e = [0,1,0,0]
l = [0,0,1,0]
o = [0,0,0,1]

In [27]:
tf.reset_default_graph()
with tf.variable_scope('one_cell') as scope:
    #one cell rnn input dimension: 4, output dimension: 2
    #one cell rnn --> rnn 한개 있는 것.
    
    hidden_size = 2
    cell = tf.contrib.rnn.BasicLSTMCell(num_units = hidden_size)

    x_data = np.array([[[1,0,0,0]]], dtype = np.float32)
    print(x_data.shape)
    pp.pprint(x_data)
    #[[[1,0,0,0]]] -> shape = (1,5,4)
    #여기서 4: input dimension(h, e, l, o 네 개)
    #여기서 5: sequence data를 몇 개 받을건지. (sequence length)
    #즉, 한 번에 sequence를 몇 개 줄 것인가, cell을 몇 번 펼칠 것인가
    #이 값은 입력 데이터의 모양에 따라 결정된다.
    
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype = tf.float32)
    sess = tf.InteractiveSession()
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

(1, 1, 4)
array([[[1., 0., 0., 0.]]], dtype=float32)
array([[[ 0.00355523, -0.1503867 ]]], dtype=float32)


In [28]:
with tf.variable_scope('two_sequences') as scope:
    hidden_size = 2
    cell = tf.contrib.rnn.BasicLSTMCell(num_units = hidden_size)
    x_data = np.array([[h,e,l,l,o]], dtype = np.float32)
    print('x_data_shape: ',x_data.shape)
    pp.pprint(x_data)
    
    outputs, states = tf.nn.dynamic_rnn(cell, x_data, dtype = tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

x_data_shape:  (1, 5, 4)
array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]], dtype=float32)
array([[[ 0.04764447, -0.11606246],
        [ 0.1670738 , -0.14869498],
        [ 0.2387336 , -0.18452227],
        [ 0.2867809 , -0.18879871],
        [ 0.21342354, -0.14986561]]], dtype=float32)


In [29]:
with tf.variable_scope('3_batches') as scope:

    x_data = np.array([[h,e,l,l,o],
                      [e,o,l,l,l],
                      [l,l,e,e,l]], dtype = np.float32)

    pp.pprint(x_data)

    cell = tf.contrib.rnn.BasicLSTMCell(num_units = 2, state_is_tuple = True)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype = tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.]],

       [[0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.]]], dtype=float32)
array([[[-0.14987193, -0.12842155],
        [-0.00043879,  0.01391789],
        [-0.03698518,  0.01553839],
        [-0.06392453,  0.02511046],
        [-0.2218964 ,  0.07190765]],

       [[ 0.09229773,  0.08715136],
        [-0.1055175 ,  0.07921923],
        [-0.08955839,  0.07433946],
        [-0.1001969 ,  0.07164992],
        [-0.1078767 ,  0.07164979]],

       [[-0.03747954,  0.00900233],
        [-0.06480017,  0.02101735],
        [ 0.0537491 ,  0.11297225],
        [ 0.14749217,  0.12718605],
        [ 0.12386174,  0.04752275]]], dtype=float32)


In [30]:
with tf.variable_scope('3_batches_dynamic_length') as scope:
    x_data = np.array([[h, e, l, l, o],
                       [e, o, l, l, l],
                       [l, l, e, e, l]], dtype=np.float32)
    pp.pprint(x_data)
    
    hidden_size = 2
    cell = tf.nn.rnn_cell.LSTMCell(num_units=hidden_size, state_is_tuple=True)
    outputs, _states = tf.nn.dynamic_rnn(
        cell, x_data, sequence_length=[5,3,4], dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())


array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.]],

       [[0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.]]], dtype=float32)
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
array([[[-1.2612730e-01,  8.3322637e-02],
        [-1.6796252e-01, -2.0496735e-02],
        [-1.7323565e-01,  1.1057492e-01],
        [-1.4439484e-01,  1.8403427e-01],
        [-1.2446473e-01,  1.3608755e-04]],

       [[-7.9471253e-02, -6.3861795e-02],
        [-9.7160198e-02, -1.1723594e-01],
        [-1.1668278e-01,  4.6641625e-02],
        [ 0.0000000e+00,  0.

In [31]:
with tf.variable_scope('initial_state') as scope:
    batch_size = 3
    x_data = np.array([[h, e, l, l, o],
                      [e, o, l, l, l],
                      [l, l, e, e, l]], dtype=np.float32)
    pp.pprint(x_data)
    
    hidden_size=2
    cell = tf.nn.rnn_cell.LSTMCell(num_units=hidden_size, state_is_tuple=True)
    initial_state = cell.zero_state(batch_size, tf.float32)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data,
                                         initial_state=initial_state, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.]],

       [[0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.]]], dtype=float32)
array([[[-0.00565487,  0.16341545],
        [ 0.13281476,  0.05420741],
        [ 0.20780823, -0.01334157],
        [ 0.26564708, -0.08057699],
        [ 0.23233508, -0.1083712 ]],

       [[ 0.15840967, -0.08561313],
        [ 0.12692918, -0.14104354],
        [ 0.21035542, -0.19514321],
        [ 0.2777927 , -0.2485614 ],
        [ 0.32509598, -0.28876027]],

       [[ 0.13008049, -0.09080652],
        [ 0.22014198, -0.16306402],
        [ 0.3131719 , -0.19464675],
        [ 0.36501324, -0.23998068],
        [ 0.37122464, -0.30883712]]], dtype=float32)


In [32]:
batch_size = 3
sequence_length = 5
input_dim = 3

x_data = np.arange(45, dtype = np.float32).reshape(batch_size, sequence_length, input_dim)
pp.pprint(x_data) 

array([[[ 0.,  1.,  2.],
        [ 3.,  4.,  5.],
        [ 6.,  7.,  8.],
        [ 9., 10., 11.],
        [12., 13., 14.]],

       [[15., 16., 17.],
        [18., 19., 20.],
        [21., 22., 23.],
        [24., 25., 26.],
        [27., 28., 29.]],

       [[30., 31., 32.],
        [33., 34., 35.],
        [36., 37., 38.],
        [39., 40., 41.],
        [42., 43., 44.]]], dtype=float32)


In [33]:
with tf.variable_scope('generated_data') as scope:
    
    cell = tf.nn.rnn_cell.LSTMCell(num_units = 5, state_is_tuple = True)
    initial_state = cell.zero_state(batch_size, tf.float32)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data,
                                         initial_state = initial_state, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

array([[[ 1.66037679e-02, -2.46753007e-01,  4.75297309e-02,
          8.80691409e-03, -5.71618602e-02],
        [ 1.55110121e-01, -4.47484255e-01,  6.51561692e-02,
         -1.95495918e-01,  4.99145836e-02],
        [ 3.90323043e-01, -5.56189179e-01,  2.37230975e-02,
         -1.92281500e-01,  8.76886770e-02],
        [ 5.28401434e-01, -6.12661362e-01,  7.79733201e-03,
         -1.47370383e-01,  6.51775375e-02],
        [ 6.12076044e-01, -6.52852893e-01,  2.40116403e-03,
         -1.06887035e-01,  4.06069830e-02]],

       [[ 7.05376267e-01, -3.93559933e-01,  6.52650022e-04,
         -5.51764965e-02,  1.35296546e-02],
        [ 7.11186707e-01, -6.28901124e-01,  2.03977790e-04,
         -4.71507907e-02,  1.23893209e-02],
        [ 7.27066815e-01, -7.16136813e-01,  6.01733154e-05,
         -3.38830687e-02,  8.04579444e-03],
        [ 7.39887714e-01, -7.58239448e-01,  1.74726774e-05,
         -2.31397562e-02,  4.75462340e-03],
        [ 7.48353243e-01, -7.85638094e-01,  5.05055959e-06,
  

In [37]:
with tf.variable_scope('dynamic_rnn') as scope:
    cell = tf.nn.rnn_cell.LSTMCell(num_units=5, state_is_tuple=True)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32,
                                         sequence_length=[1, 3, 2])
    
    print("dynamic rnn: ", outputs)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

dynamic rnn:  Tensor("dynamic_rnn/rnn/transpose_1:0", shape=(3, 5, 5), dtype=float32)
array([[[ 2.3218700e-01,  3.4416225e-02,  2.1790589e-01, -3.3154018e-02,
          8.1732064e-02],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
          0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
          0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
          0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
          0.0000000e+00]],

       [[ 1.6687806e-03,  9.0346832e-05,  7.4252528e-01, -7.2952360e-01,
          5.1477957e-05],
        [ 2.4417765e-03,  3.0845236e-05,  9.5388693e-01, -9.5589495e-01,
          6.0810766e-05],
        [ 2.6955127e-03,  6.6580337e-06,  9.9020976e-01, -9.9369395e-01,
          6.2314670e-05],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
          0.0000000e+00],
        [ 0.0000000e+

In [38]:
with tf.variable_scope('bi-directional') as scope:
    cell_fw = tf.nn.rnn_cell.LSTMCell(num_units=5, state_is_tuple=True)
    cell_bw = tf.nn.rnn_cell.LSTMCell(num_units=5, state_is_tuple=True)

    outputs, states = tf.nn.bidirectional_dynamic_rnn(cell_fw, cell_bw, x_data,
                                                      sequence_length=[2, 3, 1],
                                                      dtype=tf.float32)

    sess.run(tf.global_variables_initializer())
    pp.pprint(sess.run(outputs))
    pp.pprint(sess.run(states))

Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
(   array([[[ 6.42623231e-02,  5.81865683e-02, -2.50511467e-02,
         -1.47800535e-01, -5.76239005e-02],
        [ 4.97882694e-01,  1.23407384e-02, -3.87055539e-02,
         -3.58253717e-01, -9.70039237e-03],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00]],

       [[ 7.30319858e-01,  1.56992741e-07, -3.45836190e-04,
         -3.73301834e-01, -5.21764298e-10],
        [ 8.31203401e-01,  0.00000000e+00, -1.05899286e-04,
         -3.75248283e-01, -7.14043685e-11],
        [ 8.30586135e-01,  0.00000000e+00, -3.03309007e-05,
         -3.74288261e-01, -8.28997634e-12],
        [ 0.00000000e+00,  0.00000000e

In [39]:
hidden_size=3
sequence_length=5
batch_size=3
num_classes=5

pp.pprint(x_data)
x_data = x_data.reshape(-1, hidden_size)
pp.pprint(x_data)

softmax_w = np.arange(15, dtype=np.float32).reshape(hidden_size, num_classes)
outputs = np.matmul(x_data, softmax_w)
outputs = outputs.reshape(-1, sequence_length, num_classes)
pp.pprint(outputs)

array([[[ 0.,  1.,  2.],
        [ 3.,  4.,  5.],
        [ 6.,  7.,  8.],
        [ 9., 10., 11.],
        [12., 13., 14.]],

       [[15., 16., 17.],
        [18., 19., 20.],
        [21., 22., 23.],
        [24., 25., 26.],
        [27., 28., 29.]],

       [[30., 31., 32.],
        [33., 34., 35.],
        [36., 37., 38.],
        [39., 40., 41.],
        [42., 43., 44.]]], dtype=float32)
array([[ 0.,  1.,  2.],
       [ 3.,  4.,  5.],
       [ 6.,  7.,  8.],
       [ 9., 10., 11.],
       [12., 13., 14.],
       [15., 16., 17.],
       [18., 19., 20.],
       [21., 22., 23.],
       [24., 25., 26.],
       [27., 28., 29.],
       [30., 31., 32.],
       [33., 34., 35.],
       [36., 37., 38.],
       [39., 40., 41.],
       [42., 43., 44.]], dtype=float32)
array([[[  25.,   28.,   31.,   34.,   37.],
        [  70.,   82.,   94.,  106.,  118.],
        [ 115.,  136.,  157.,  178.,  199.],
        [ 160.,  190.,  220.,  250.,  280.],
        [ 205.,  244.,  283.,  322.,  361.]],

 

In [42]:
y_data = tf.constant([[1, 1, 1]])
prediction = tf.constant([[[0.2, 0.7], [0.6, 0.2], [0.2, 0.9]]], dtype=tf.float32)

weights = tf.constant([[1, 1, 1]], dtype=tf.float32)

sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=prediction, targets=y_data, weights=weights)
sess.run(tf.global_variables_initializer())
print("Loss: ", sequence_loss.eval())

Loss:  0.5967595


In [41]:
y_data = tf.constant([[1, 1, 1]])

prediction1 = tf.constant([[[0.3, 0.7], [0.3, 0.7], [0.3, 0.7]]], dtype=tf.float32)
prediction2 = tf.constant([[[0.1, 0.9], [0.1, 0.9], [0.1, 0.9]]], dtype=tf.float32)

prediction3 = tf.constant([[[1, 0], [1, 0], [1, 0]]], dtype=tf.float32)
prediction4 = tf.constant([[[0, 1], [1, 0], [0, 1]]], dtype=tf.float32)

weights = tf.constant([[1, 1, 1]], dtype=tf.float32)

sequence_loss1 = tf.contrib.seq2seq.sequence_loss(prediction1, y_data, weights)
sequence_loss2 = tf.contrib.seq2seq.sequence_loss(prediction2, y_data, weights)
sequence_loss3 = tf.contrib.seq2seq.sequence_loss(prediction3, y_data, weights)
sequence_loss4 = tf.contrib.seq2seq.sequence_loss(prediction3, y_data, weights)

sess.run(tf.global_variables_initializer())
print("Loss1: ", sequence_loss1.eval(),
      "Loss2: ", sequence_loss2.eval(),
      "Loss3: ", sequence_loss3.eval(),
      "Loss4: ", sequence_loss4.eval())

Loss1:  0.5130153 Loss2:  0.3711007 Loss3:  1.3132616 Loss4:  1.3132616
